<a href="https://colab.research.google.com/github/Neetagrg/Mini-Project_2_Spring-2025-WEB-DATA-MINING-CUS-635/blob/main/mini_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project 2: Neeta Kumari & Bir Bahadur Gharti Class CUS635

# Install Libraries

In [310]:
!pip install boto3 requests pandas


# Import Libraries

In [311]:
import boto3
from botocore.config import Config
from botocore import UNSIGNED
import pandas as pd
import requests
import json


# Set Up AWS S3 Client

In [312]:
# constants
TEAM_FOLDER = "TEAM_6/"
BUCKET_NAME = "cus635-spring2025"

# Anonymous S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))


# Download a File from S3

In [313]:
# file path in S3
filename = TEAM_FOLDER + "Financial_Articles.csv"

# Download the file and save locally
s3.download_file(BUCKET_NAME, filename, "local_copy.csv")

print("Download complete: local_copy.csv")


Download complete: local_copy.csv


In [314]:
# Load the CSV into a DataFrame
df = pd.read_csv("local_copy.csv")

# Display the first 5 rows
df.head()


,heading,context
0,Motor finance compensation ruling ‘goes too fa...,<p>A court of appeal ruling that has left lend...
1,Supreme court judges reject Reeves’ motor fina...,<p>Rachel Reeves was dealt a fresh blow on Mon...
2,Car finance firm Close Brothers slumps to loss...,"<p>Close Brothers, one of the UK’s biggest pro..."
3,Chief finance officer of under-fire Thames Wat...,<p>Thames Water’s chief financial officer has ...
4,Car finance scandal: lender sets aside £165m f...,<p>One of the UK’s biggest providers of motor ...


# DocumentProcessor class

In [315]:
!pip install pinecone sentence-transformers tokenizers


In [316]:
!pip install pinecone
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding
from tokenizers import Tokenizer
from google.colab import userdata


# Initialize the Embedding Model

Used pre-trained model for embedding

In [317]:
# Load the BERT tokenizer and the model
tokenizer = Tokenizer.from_pretrained("google-bert/bert-base-uncased")
static_embedding = StaticEmbedding(tokenizer, embedding_dim=1024)

# Create the sentence transformer model using the static embedding
model = SentenceTransformer(modules=[static_embedding])

# Test the model with a sample sentence
model.encode("This is a test sentence.")



array([-0.09519723,  0.32112613,  0.21214421, ...,  0.4887556 ,
       -0.6722111 ,  0.03288254], dtype=float32)

# Connect to Pinecone

In [318]:
import os
from pinecone import Pinecone

# Pinecone API key
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6")

# Connect to index through the Pinecone client instance
index = pc.Index("cus635")



# Create Finance Investment Articles

In [319]:
# Define finance-related investment articles
investment_articles = [
    {"id": "vec1", "text": "Investing in stocks can offer significant returns over time, but it comes with risks. It is important to diversify your portfolio and consider both short-term and long-term goals."},
    {"id": "vec2", "text": "Bonds are a safer investment option than stocks and can provide a steady income stream. However, they tend to offer lower returns, especially in a low-interest-rate environment."},
    {"id": "vec3", "text": "Real estate investments provide a tangible asset and can generate both income and capital appreciation. However, real estate markets can be volatile and require significant upfront capital."},
    {"id": "vec4", "text": "Cryptocurrency is an emerging investment vehicle, known for its high volatility. While it can offer substantial rewards, it is also associated with significant risk and regulatory uncertainty."},
    {"id": "vec5", "text": "Mutual funds pool money from multiple investors to invest in a diversified portfolio of stocks, bonds, or other securities. They provide a way for small investors to access diversified investments."},
    {"id": "vec6", "text": "Exchange-Traded Funds (ETFs) are similar to mutual funds but are traded on stock exchanges like individual stocks. They offer liquidity and diversification in a single investment."},
    {"id": "vec7", "text": "Index funds are a type of mutual fund or ETF that aims to replicate the performance of a market index, like the S&P 500. They are often considered low-cost and long-term investment options."},
    {"id": "vec8", "text": "Private equity involves investing in private companies, often through venture capital or buyouts. While private equity investments can yield high returns, they are typically illiquid and high-risk."},
    {"id": "vec9", "text": "Hedge funds use various strategies to generate returns, including short selling, leverage, and derivatives. They are typically accessible only to accredited investors and involve higher fees."},
    {"id": "vec10", "text": "Commodity investments, such as gold, oil, and agricultural products, provide a way to hedge against inflation and diversify a portfolio. These investments are often volatile, reflecting changes in global supply and demand."}
]


# Embed Documents

In [320]:
# Embed the documents (investment articles)
embeddings = model.encode([article['text'] for article in investment_articles])


# Initialize Pinecone and Insert Data

In [325]:
from pinecone import Pinecone

# Connect to Pinecone
pc = Pinecone(api_key="pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6")

index_name = 'cus635'

# Connect to the existing index
index = pc.Index(index_name)
print(f"Connected to existing index: {index_name}")

# Prepare vectors (assuming embeddings is a list of embeddings with correct dimension)
vectors = []
for i, embedding in enumerate(embeddings):
    vectors.append((
        str(i),
        embedding.tolist(),
        {
            'text': investment_articles[i]['text'],
            'category': 'investment'
        }
    ))

# Upload (upsert) the vectors into Pinecone under namespace 'team 6'
index.upsert(vectors=vectors, namespace='team 6')

print(f"{len(vectors)} investment articles have been uploaded to Pinecone under 'team 6'.")


Connected to existing index: cus635


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 28 Apr 2025 22:32:13 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '321', 'x-pinecone-request-id': '4851080155939276947', 'x-envoy-upstream-service-time': '80', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 1024 does not match the dimension of the index 384","details":[]}


# Query the Pinecone Index

In [ ]:
# Query: Search for articles related to "stocks"
query = "stocks"
query_vector = model.encode(query).tolist()

# Perform the query in Pinecone
results = index.query(
    vector=query_vector,
    top_k=5,  # Number of top results to return
    include_metadata=True,
    namespace="finance"
)

# Print the top results
print(f"\nTop results for query: \"{query}\"\n")
for i, match in enumerate(results['matches']):
    print(f"[{i}] Score: {match['score']:.3f}")
    print(f"Text: {match['metadata']['text']}\n")



Top results for query: "stocks"



In [ ]:
#Perform the query
query_text = "investment opportunities in AI"
query_model = SentenceTransformer('all-MiniLM-L6-v2')

# Query text to a vector using the model
query_vector = query_model.encode([query_text])[0]

# Numpy array to a list
query_vector = query_vector.tolist()

# Query Pinecone for the top 10 most similar articles
results = index.query(
    vector=query_vector,
    top_k=10,
    include_metadata=True,
    namespace="team 6"
)

# Output the results
print(f"Found {len(results['matches'])} similar articles for query: '{query_text}'")

for match in results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    print(f"Text: {match['metadata']['text']}\n")


Found 10 similar articles for query: 'investment opportunities in AI'
ID: 9, Score: 0.304798186
Text: Commodity investments, such as gold, oil, and agricultural products, provide a way to hedge against inflation and diversify a portfolio. These investments are often volatile, reflecting changes in global supply and demand.

ID: 0, Score: 0.302832276
Text: Investing in stocks can offer significant returns over time, but it comes with risks. It is important to diversify your portfolio and consider both short-term and long-term goals.

ID: 7, Score: 0.261151761
Text: Private equity involves investing in private companies, often through venture capital or buyouts. While private equity investments can yield high returns, they are typically illiquid and high-risk.

ID: 1, Score: 0.260233402
Text: Bonds are a safer investment option than stocks and can provide a steady income stream. However, they tend to offer lower returns, especially in a low-interest-rate environment.

ID: 2, Score: 0.256